## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-31-10-47-59 +0000,bbc,Guinea junta chief wins presidential election ...,https://www.bbc.com/news/articles/cx2520nyp8yo...
1,2025-12-31-10-47-36 +0000,nyt,Justice Dept. Is Now Said to Be Reviewing 5.2 ...,https://www.nytimes.com/2025/12/30/us/politics...
2,2025-12-31-10-30-00 +0000,wsj,Warren Buffett Stayed True to His Ways in His ...,https://www.wsj.com/finance/warren-buffett-sta...
3,2025-12-31-10-30-00 +0000,wsj,Six Stories That Show How Trump’s Trade War Ch...,https://www.wsj.com/economy/trade/trump-trade-...
4,2025-12-31-10-30-00 +0000,wsj,Instead of crumbling under the highest U.S. ta...,https://www.wsj.com/economy/trade/trump-trade-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2358/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,22
20,year,14
44,new,12
322,minnesota,11
418,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
81,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...,76
118,2025-12-30-23-13-27 +0000,nypost,Trump admin freezes all childcare payments to ...,https://nypost.com/2025/12/30/us-news/trump-ad...,70
122,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...,67
257,2025-12-30-12-00-00 +0000,nypost,Minnesota Gov. Tim Walz was warned of rampant ...,https://nypost.com/2025/12/30/us-news/minnesot...,66
44,2025-12-31-05-43-03 +0000,startribune,Trump administration says it's freezing child ...,https://www.startribune.com/trump-administrati...,63


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
81,76,2025-12-31-02-27-00 +0000,wsj,The Trump administration said it is freezing f...,https://www.wsj.com/politics/policy/trump-admi...
122,67,2025-12-30-23-05-25 +0000,cbc,Israel will ban dozens of aid groups from Gaza...,https://www.cbc.ca/radio/asithappens/gaza-aid-...
47,35,2025-12-31-05-20-55 +0000,nyt,"Tatiana Schlossberg, Kennedy Daughter Who Wrot...",https://www.nytimes.com/2025/12/30/us/politics...
184,35,2025-12-30-19-35-00 +0000,wsj,Some Federal Reserve officials said they were ...,https://www.wsj.com/economy/central-banking/fe...
4,35,2025-12-31-10-30-00 +0000,wsj,Instead of crumbling under the highest U.S. ta...,https://www.wsj.com/economy/trade/trump-trade-...
223,33,2025-12-30-16-55-56 +0000,nypost,"Meta snaps up AI startup Manus for $2B, drawin...",https://nypost.com/2025/12/30/business/meta-sn...
217,33,2025-12-30-17-13-49 +0000,nypost,DC pipe bomb suspect Brian Cole Jr. is autisti...,https://nypost.com/2025/12/30/us-news/dc-pipe-...
82,31,2025-12-31-02-19-51 +0000,nypost,Netanyahu warns ‘all bets are off’ if Iran see...,https://nypost.com/2025/12/30/us-news/netanyah...
205,30,2025-12-30-18-18-44 +0000,nypost,Zohran Mamdani names two controversial lawyers...,https://nypost.com/2025/12/30/us-news/zohran-m...
218,29,2025-12-30-17-12-11 +0000,bbc,New video appears to show Texas teen missing s...,https://www.bbc.com/news/articles/cdreljlx21mo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
